In [1]:
import os
import pandas as pd
import geopandas as gpd

gtfs_folder = r"gtfs"

routes_path = os.path.join(gtfs_folder, "routes.txt")
trips_path = os.path.join(gtfs_folder, "trips.txt")
shapes_path = os.path.join(gtfs_folder, "shapes.txt")
stops_path = os.path.join(gtfs_folder, "stops.txt")
stop_times_path = os.path.join(gtfs_folder, "stop_times.txt")
calendar_path = os.path.join(gtfs_folder, "calendar.txt")
calendar_dates_path = os.path.join(gtfs_folder, "calendar_dates.txt")
fare_attributes_path = os.path.join(gtfs_folder, "fare_attributes.txt")

routes_df = pd.read_csv(routes_path, dtype=str)
trips_df = pd.read_csv(trips_path, dtype=str)
shapes_df = pd.read_csv(shapes_path, dtype=str)
stops_df = pd.read_csv(stops_path, dtype=str)
stop_times_df = pd.read_csv(stop_times_path, dtype=str)
fare_attributes_df = pd.read_csv(fare_attributes_path, dtype=str)


In [2]:
# Convert stops to a GeoDataFrame
# Ensure latitude/longitude are numeric and drop invalid rows
stops_df['stop_lat'] = pd.to_numeric(stops_df.get('stop_lat'), errors='coerce')
stops_df['stop_lon'] = pd.to_numeric(stops_df.get('stop_lon'), errors='coerce')
stops_with_coords = stops_df.dropna(subset=['stop_lat', 'stop_lon']).copy()

gdf_stops = gpd.GeoDataFrame(
    stops_with_coords,
    geometry=gpd.points_from_xy(stops_with_coords['stop_lon'], stops_with_coords['stop_lat']),
    crs='EPSG:4326'
)

gdf_stops.drop(columns=['stop_lat', 'stop_lon'], inplace=True)
# Reset index for a clean GeoDataFrame
gdf_stops = gdf_stops.reset_index(drop=True)
# Show the first rows and CRS
gdf_stops.head()
print('CRS:', gdf_stops.crs)

CRS: EPSG:4326


In [3]:
gdf_stops.head()

,stop_id,stop_name,geometry
0,MACRO_001_001,Fray Angélico,POINT (-103.34283 20.60824)
1,MACRO_001_002,Esculturas,POINT (-103.34277 20.61182)
2,MACRO_001_003,Artes Plásticas,POINT (-103.3434 20.61629)
3,MACRO_001_004,Clemente Orozco,POINT (-103.34669 20.6224)
4,MACRO_001_005,López De Legazpi,POINT (-103.34838 20.62731)


In [7]:
def openMap(gdf):
    m = gdf.explore(
    tiles="https://tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr="OSM"
    )
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [8]:
gdf_stops_with_J = gdf_stops[gdf_stops['stop_name'].str.startswith('J')]
openMap(gdf_stops_with_J)